In [10]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from difflib import SequenceMatcher as SM
import pandas as pd
import numpy as np
import time
import Levenshtein
import re
from newsapi import NewsApiClient
import requests
import bs4 as bs
from bs4.element import Comment
from IPython.display import display,HTML
from newsapi import NewsApiClient

In [ ]:
#!pip install newsapi-python

In [14]:
data = pd.read_csv('dataXData.csv')
company = data['Business Name'].unique()
company = company.astype(str)
#data.head()

In [15]:
pd.unique(data[['City']].values.ravel())

array([nan, 'quebec', 'bc', ..., 'temperance', 'delmont', 'monaca'],
      dtype=object)

In [16]:
def nameCleaner(name): 
    name = name.lower()
    name = name.split()
    #eng_stopwords = set(stopwords.words("english"))
    #name = [w for w in name if not w in eng_stopwords]
    company_stopwords = set(['group', 'company', 'limited', 'cos', 'dba', 'lllp', 'ltd', 'sa', 'ag', 'pllc', 'pl', 'nv', 'ltcc', 'bv', 'gmblt', 'llc', 'sia', 'sta', 'inc', 'co', 'corp', 'pte'])
    a= []
    for w in name:
        w = re.sub('[^a-zA-Z0-9]+', '',w)
        if(w not in company_stopwords):
            a.append(w)
    name = ''.join(a)
    return name

In [17]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [13]:
start=time.time()
response='y'
while(response=='y'):
    a=[]
    text2=input("Which text do you want to search? ")
    text1 = nameCleaner(text2)
    company = data['Search Term'].unique()
    company = company.astype(str)
    for i in range(0,len(company),1):
        search=company[i].lower()
        a.append(fuzz.token_sort_ratio(text1,search))

    #Finding the company names with maximum risk score aka fuzzy ratio
    ratio=np.array(a)
    c=np.where(ratio==ratio.max())
    maximum=max(a)
    top = company[c]
    top_list = set(data.loc[data['Search Term'].isin(top)]['Business Name'])
    print("Risk Score for", color.BOLD +text2 + color.END,"is",color.BOLD + str(maximum) + color.END,"for the brands",top_list)
    
    #Decision Logic for Classification
    if maximum>=90:
        print(color.BOLD,color.RED +"High risk"+ color.END)
    elif (maximum<90 and maximum>=60):
        print(color.BOLD,color.YELLOW +"Medium Risk"+ color.END)
    else:
        print(color.BOLD,color.GREEN +"No Risk"+ color.END)

    # News API part begins here
    df=data.loc[data['Search Term'].isin(top)][['Business Name','Industry','City','State']]
    df1=df.drop_duplicates(keep='first')
    display(df1)
    unique_names = []
    for d in top_list:
        lower = d.lower()
        if lower not in unique_names:
            unique_names.append(lower)

    #print(unique_names)

    def text_from_html(body):
        soup = bs.BeautifulSoup(body, 'html.parser')
        texts = soup.findAll(text=True)
        visible_texts = filter(tag_visible, texts)  
        # encode to get rid of encoding errors (hopefully)
        return " ".join(t.strip() for t in visible_texts).encode('utf-8','ignore')

    def tag_visible(element):
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        return True

    entity_column = []
    url_column = []
    snippet_column = []
    # Init
    newsapi = NewsApiClient(api_key='a28acbbe0b2a46428906f353d180c5c3')
    all_articles = []
    for entity in unique_names:
        all_articles=newsapi.get_everything(q=entity,
                                             language='en', # necessary to only maintain english? probably for now
                                             sort_by='relevancy',
                                             page=1)["articles"]

        for article in all_articles:
            url = article["url"]        
            source = requests.get(url)
            if source.status_code != 200:
                continue
            text = text_from_html(source.content).decode('utf-8')

            if (' ' not in entity):
                splitted = text.split()
                lower = [x.lower() for x in splitted]

                try:
                    index = lower.index(entity);

                except ValueError:
                    index = -1
                if(index != -1):
                    if(index < 50):
                        article_snippet = " ".join(splitted[0:index+50])
                    else:
                        article_snippet = " ".join(splitted[index-50:index+50])
                    entity_column.append(entity)
                    url_column.append(url)
                    snippet_column.append(article_snippet)
            else:
                indices = [m.start() for m in re.finditer(entity.lower(), text.lower())]

                if not indices: 
                    continue
                index = indices[0]
                if(index < 100):
                    article_snippet = text[0:index+100]
                else:
                    article_snippet = text[index-100:index+100]
                entity_column.append(entity)
                url_column.append(url)
                snippet_column.append(article_snippet)


    columns = {"Entity" : entity_column, "URL": url_column, "Snippet" : snippet_column}
    df_result = pd.DataFrame.from_dict(columns)
    pd.set_option('max_colwidth', 300)
    display(df_result) 
    
    #Recording time taken to search the database and NEWS API
    end=time.time()
    print("Time taken:",round(end-start,0),"seconds")
    response=input("Do you want to search again? [y/n] ")
print(color.BOLD + "THANK YOU!, for searching with us!" + color.END)

Which text do you want to search? johnson
Risk Score for johnson is 100 for the brands {'JOHNSON'}
 High risk


,Business Name,Industry,City,State
861702,JOHNSON,Journeyman Electrician,tx,TX
862019,JOHNSON,Cosmetology Operator Instructor,tx,TX
862438,JOHNSON,Cosmetology Beauty Salon,tx,TX
862802,JOHNSON,Apprentice Electrician,tx,TX
862955,JOHNSON,Cosmetology Operator,tx,TX
863435,JOHNSON,A/C Contractor,tx,TX
863486,JOHNSON,Tow Truck Operator (Consent Tow),tx,TX
863925,JOHNSON,"Barber, Class A",tx,TX
864938,JOHNSON,Cosmetology Manicurist,tx,TX
865433,JOHNSON,Cosmetology Esthetician,tx,TX


,Entity,URL,Snippet
0,johnson,https://lifehacker.com/im-mars-showrunner-dee-johnson-and-this-is-how-i-work-1830082959,"Toggle Conversation tools Go to permalink Photo: National Geographic/Dusan Martincek How I Work The How I Work series asks heroes, experts, and flat-out productive people to share their shortcuts, workspaces, routines, and more. Have someone you want to see featured, or questions you think we sh..."
1,johnson,https://www.nytimes.com/2018/11/10/obituaries/ron-johnson-dead-giants.html,"Sections SEARCH Skip to content Skip to site index Obituaries Subscribe Log In Log In Today’s Paper Obituaries | Ron Johnson, Record-Setting Giants Running Back, Dies at 71 Supported by Ron Johnson, Record-Setting Giants Running Back, Dies at 71 Image Johnson in 1970, his first season with the G..."
2,johnson,https://www.nytimes.com/2018/11/21/opinion/trump-clinton-mcgann-prosecution.html,"only thing blocking his plan to prosecute Hillary Clinton was Don McGahn. Think about that. By Harry Litman Mr. Litman is a former United States attorney and deputy assistant attorney general. Nov. 21, 2018 Image White House counsel Don McGahn looks on as President Trump prepares to pardon Boxer..."
3,johnson,https://www.nytimes.com/2018/11/21/us/homeless-kansas-city-food-bleached.html,Sections SEARCH Skip to content Skip to site index U.S. Subscribe Log In Log In Today’s Paper U.S. | You Want to Feed the Hungry? Lovely. Let’s See Your Permit. Supported by You Want to Feed the Hungry? Lovely. Let’s See Your Permit. Image Richard Johnson received a cup of hot gumbo from a Free ...
4,johnson,https://www.nytimes.com/2018/11/30/arts/design/post-modernism-architect.html,"to reconstruct it. The city‘s cost-conscious solution was a respectful renovation, now underway, that is interpretatively restoring Post-Modernism’s first declarative monument. Debates in Portland anticipated debates about preserving the movement’s progeny elsewhere. Back in 1984, two years afte..."
5,johnson,https://www.theguardian.com/business/2018/nov/16/patisserie-valerie-chair-hires-former-sfo-investigator-as-his-lawyer,switch to the Australia edition switch to the International edition Search jobs Digital Archive The Guardian app Video Podcasts Pictures Newsletters Inside the Guardian Guardian Weekly Crosswords Facebook Twitter Search jobs Digital Archive Patisserie Valerie Patisserie Valerie chair hires forme...
6,johnson,https://gizmodo.com/runaway-peacock-ditches-his-human-family-for-new-turkey-1830635569,His Human Family for New Turkey Friends Catie Keck 11/24/18 7:45pm Filed to: peacock on the run Filed to: peacock on the run peacock on the run Animals Vermont peacock Turkey 45 3 Edit Embiggen Send to Editors Promote Share to Kinja Toggle Conversation tools Go to permalink Photo: Rene Johnson A...
7,johnson,https://www.theguardian.com/society/2018/nov/27/vaginal-mesh-implant-sold-despite-warnings-could-cause-pain-johnson-johnson,Guardian app Video Podcasts Pictures Newsletters Inside the Guardian Guardian Weekly Crosswords Facebook Twitter Search jobs Digital Archive UK World Business Football UK politics Environment Education Society Science Tech Global development Cities Obituaries More The Implant Files Pharmaceutica...
8,johnson,https://www.theguardian.com/football/2018/nov/18/michael-johnson-guyana-manager-i-stopped-counting-on-42-rejections,The Guardian app Video Podcasts Pictures Newsletters Inside the Guardian Guardian Weekly Crosswords Facebook Twitter Search jobs Digital Archive Soccer Live scores Tables Fixtures Results Competitions Clubs World Cup 2018 More The Observer Soccer Interview Michael Johnson: ‘I stopped counting af...
9,johnson,https://www.nytimes.com/2018/11/30/us/kansas-license-plate-recall.html,"newspaper, Pacific Citizen, that there was any action. Barbara Johnson, a Kansas resident and member of a Japanese American Citizens League chapter in Omaha, Neb., who has painful memories of anti-Japanese sentiment growing up in the late 1950s, was moved 

Time taken: 27.0 seconds
Do you want to search again? [y/n] n
Thanks for searching with us!
